# Introduction

This sample notebook takes you through an end-to-end workflow to train model on Amazon Rekognition Custom Labels using data set generated by Amazon GroundTruth

In [94]:
import datetime
import tarfile
import boto3
import os
from sagemaker import get_execution_role
import sagemaker
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont, ExifTags, ImageColor
import io

## 1. Upload Images to S3

In [95]:
bucket_name = 'aws-workshops-labels-12345678' ## Update the value with the bucket name created earlier in the lab
region = boto3.Session().region_name    
s3_client = boto3.client('s3', region_name=region)

#### Skip the below step if you already have images uploaded to your S3 bucket

In [ ]:
## Uploading Licensed Images for raw data
source_dir = '../images/raw-data/LicensedImages-CreativeCommons'
dest_dir = 'raw-data/images'
file_list = os.listdir(source_dir)
for file in file_list :   
    if file != '.ipynb_checkpoints':
        response = s3_client.upload_file(source_dir+'/'+file, bucket_name, dest_dir+"/"+file)
        print (file + ' uploaded')
print('Raw Data Upload Complete to '+bucket_name+'/'+dest_dir)

## Uploading Non-Licensed Images for raw data
source_dir = '../images/raw-data/LicenseNotNeeded_Images'
dest_dir = 'raw-data/images'
file_list = os.listdir(source_dir)
s3_client = boto3.client('s3', region_name=region)
for file in file_list : 
    if file != '.ipynb_checkpoints':
        response = s3_client.upload_file(source_dir+'/'+file, bucket_name, dest_dir+"/"+file)
        print (file + ' uploaded')
print('Raw Data Upload Complete to '+bucket_name+'/'+dest_dir)

## Uploading Test Data
source_dir = '../images/test-data'
dest_dir = 'test-data/images'
file_list = os.listdir(source_dir)
s3_client = boto3.client('s3', region_name=region)
for file in file_list : 
    response = s3_client.upload_file(source_dir+'/'+file, bucket_name, dest_dir+"/"+file)
    print (file + ' uploaded')
print('Test Data Upload Complete to '+bucket_name+'/'+dest_dir)


Note: If you get the following error

<img src="../lab-images/s3error.png">

Please make sure you have updated the correct **bucket_name** with your bucket

### Let's look at one of the images

In [ ]:
imageName = "raw-data/images/800px-Woodpeckers-Telephone-Cable.jpg"
display(IImage(url=s3_client.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': imageName})))

### 2. Copy Existing Annotation data to S3 Bucket. 

### <span style="color:red">Note: If you have completed labeling all the images on GroundTruth and want to use your own labeled data set for training Rekognition model, skip Step 2</span>

#### Update S3 bucket name in existing data

In [ ]:
old_bucket_name = "aws-workshops-labels-1234567" ## DO NOT MODIFY. This value comes from existing manifest file. 
new_bucket_name = bucket_name
!echo "Occurences of old_bucket_name i.e. $old_bucket_name in original manifest file"
!grep -ir $old_bucket_name ../images/annotated-data/manifests/output/output.manifest | wc -l 
!sed -i.bak -e "s/$old_bucket_name/$new_bucket_name/g" ../images/annotated-data/manifests/output/output.manifest
!echo "Occurences of old_bucket_name i.e. $old_bucket_name in original manifest file"
!grep -ir '$old_bucket_name' ../images/annotated-data/manifests/output/output.manifest | wc -l 
!echo "Occurences of new_bucket_name i.e. $new_bucket_name in updated manifest file"
!grep -ir $new_bucket_name ../images/annotated-data/manifests/output/output.manifest | wc -l 

#### Upload annotation metadata to S3 bucket

In [ ]:
OUTPUT = 's3://{}/{}'.format(bucket_name, 'annotated-data')

## Replace bucket name in manifest file with new bucket name

## Uploading annotation data to S3 bucket
!aws s3 cp ../images/annotated-data {OUTPUT} --recursive --quiet


## 2.  Create Project in Amazon Rekognition Custom Labels

### Create Project
- On the home page, click on **Use Custom Labels** 

**Note**: *Make sure you are in same **AWS region** as S3 bucket when creating Rekognition Custom Labels project*
- On the next page, click on **Get Started** button
- If you are creating using Custom Labels for the first time in this AWS Account, you need to allow service to create S3 bucket. 
- Specify the name of the project as **aws-workshops-rekognition-custom-labels**
- Click on **Create Project** button

<img src="../lab-images/19.png" width="800">
<img src="../lab-images/20.png" width="800">
<img src="../lab-images/36.png" width="800">
<img src="../lab-images/21.png" width="800">

## 3. Create Data Set
### (Follow Screenshots below)


- Click on **Create dataset** button
- Specify name for the dataset as **aws-workshops-gt-data**
- Select Option - **Import images labeled by Amazon SageMaker Ground Truth**
- Specify the location of the output manifest file generated by SageMaker Ground Truth Labeling Job - **s3://{bucket-name}/annotated-data/manifests/output/output.manifest**
- Copy and paste the bucket policy by **copying the generated bucket policy** and **then clicking on hyperlink** in the screenshot - **Paste the policy into the "Bucket Policy" section of ...**
- Return to **Rekognition Data set creation page** and Click on **submit** button

In [ ]:
print ('S3 location of manifest file - \n' + 's3://{}/annotated-data/manifests/output/output.manifest'.format(bucket_name))


<img src="../lab-images/22.png" width="800">
<img src="../lab-images/23.png" width="800">
<img src="../lab-images/24.png" width="800">
<img src="../lab-images/25.png" width="800">
<img src="../lab-images/38.png" width="800">

#### [OPTIONAL STEP - Only needed if you want to do additional labeling or if you did not label the images previously]
#### Add Labels
- Click on **Start Labeling** button
- Click on **Add** button on the next page
- Type Label - **hole** and **no_hole** and save it by clicking on **Save** button
- Once done, click on **Exit** button to complete the labeling job

<img src="../lab-images/39.png" width="800">
<img src="../lab-images/startlabeling.png" width="800">
<img src="../lab-images/43.png" width="800">

### Train Model
- Once the dataset is created, click on **Train model** button to start training the automatically identified model

<img src="../lab-images/26.png" width="800">

### Specify dataset for training
- Select the **previously created** training dataset from the drop down in **Choose training dataset**. Example - **aws-workshops-gt-data**
- Select **Spit training dataset** to spit the data set into training and test data for model training and evaluation
- Click on **Train** button to start training model.

<img src="../lab-images/29.png" width="800">

### Training Status
**Note this will take ~1-2 hours depending upon a number of factors**

<img src="../lab-images/30.png" width="800">

### Evaluate Model and Testing Results
- Once the training is completed, click on training model link
- On the next page, you will see the model evaluation details, Look at the various values for **F1 score, Precision, Recall**. These values depend on the training data.
- Click on **View test results** button to check the results on test data

<img src="../lab-images/31.png" width="800">
<img src="../lab-images/32.png" width="800">
<img src="../lab-images/33.png" width="800">

## How to evaluate trained model
Reference Docs - https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/tr-metrics-use.html

**Precision** - Precision is the fraction of correct predictions (true positives) over all model predictions (true and false positives) at the assumed threshold for an individual label. As the threshold is increased, the model might make fewer predictions. In general, however, it will have a higher ratio of true positives over false positives compared to a lower threshold. Possible values for precision range from 0–1, and higher values indicate higher precision.

**Recall** - Recall is the fraction of your test set labels that were predicted correctly above the assumed threshold. It is a measure of how often the model can predict a custom label correctly when it's actually present in the images of your test set. The range for recall is 0–1. Higher values indicate a higher recall.

For the given business problem, you may want higher precision and lower recall. Depending upon training data set, the precision and recall values will differ. You can further improve the model accuracy by following the steps mentioned on https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/tr-improve-model.html

## Detect Holes using the trained model

To analyze an image with a trained Amazon Rekognition Custom Labels model, you call the DetectCustomLabels API. The result from DetectCustomLabels is a prediction that the image contains specific objects, scenes, or concepts.




Get values of **project_arn** , **model_arn** and **version_name** as shown in the below screenshot

- Click on Project Version once the model training is completed. It shows **TRAINING_COMPLETED**
- Click on **Use Model** tab 
- Expand **API Code** 
- Select **Python** and get the values of variables from **Start Model** code

<img src="../lab-images/44.png" width="800">
<img src="../lab-images/45.png" width="800">
<img src="../lab-images/46.png" width="800">
<img src="../lab-images/47.png" width="800">


In [96]:
# project_arn is the ARN of the Custom Labels Project
project_arn='YOUR PROJECT ARN'

# model_arn is the ARN of the Model we want to use from the project we used for custom labels
model_arn='YOUR MODEL ARN'

# version_name is the custom labeling project name (something like: aws-workshops-rekognition-custom-labels.2021-05-16T11.23.30)
version_name='YOUR PROJECT VERSION NAME'

# Put in the name of your image
photo="test-data/images/14.jpg" ## NO NEED TO CHANGE


Before we can run the mode we need to start the inference end point with the following code.

In [ ]:
def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def startmodel():
    min_inference_units=1 
    start_model(project_arn, model_arn, version_name, min_inference_units)
    
startmodel();

In this section we setup the code that will allow us to analyze the image, but we won't start the analysis yet.

In [116]:
def display_image(bucket,photo,response):
    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

    # Ready image to draw bounding boxes on it.
    imgWidth, imgHeight = image.size
    draw = ImageDraw.Draw(image)
    lcount = 0
    # calculate and display bounding boxes for each detected custom label
    print('Detected custom labels for ' + photo)
    for customLabel in response['CustomLabels']:
        if 'Geometry' in customLabel and customLabel['Name'] == 'hole' :
            box = customLabel['Geometry']['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']
            lcount = lcount + 1
            print ("-------------------------------------------------")
            print ('Label ' + str(lcount) + ":")
            print('Label Name :' + str(customLabel['Name']))
            print('Confidence ' + str(customLabel['Confidence']))
            # fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', 50)
            fnt = ImageFont.truetype('/usr/share/fonts/dejavu/DejaVuSans.ttf', 50)
            
            draw.text((left,top), customLabel['Name'], fill='#00d400', font=fnt)

            print('Left: ' + '{0:.0f}'.format(left))
            print('Top: ' + '{0:.0f}'.format(top))
            print('Label Width: ' + "{0:.0f}".format(width))
            print('Label Height: ' + "{0:.0f}".format(height))
            print ("-------------------------------------------------")
            
            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            draw.line(points, fill='#00d400', width=5)

        #image.show()
    display(image)

def show_custom_labels(model,bucket,photo, min_confidence):
    client=boto3.client('rekognition')

    #Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)

    # For object detection use case, uncomment below code to display image.
    display_image(bucket,photo,response)
    count =0
    for labels in response['CustomLabels']:
        if labels ['Name'] == 'hole':
            count = count + 1
    return count

def analyzeImage():
    min_confidence=85
    
    label_count=show_custom_labels(model_arn,bucket_name,photo, min_confidence)
    print("Number of Holes Detected: " + str(label_count)) 

In the final section we run the analysis of our image using the model we trained. 

In [ ]:
analyzeImage();

Once we completed out analysis we can shut down the Model endpoint with the following code

In [ ]:
def stop_model(model_arn):

    client=boto3.client('rekognition')

    print('Stopping model:' + model_arn)

    #Stop the model
    try:
        response=client.stop_project_version(ProjectVersionArn=model_arn)
        status=response['Status']
        print ('Status: ' + status)
    except Exception as e:  
        print(e)  

    print('Done...')
    
def stopModel():
    stop_model(model_arn)
    
stopModel();

# Review

We covered a lot of ground in this notebook! Let's recap what we accomplished. First we uploaded the labeled data set generated by SageMaker Ground Truth labeling job to S3 bucket. We then trained model in Amazon Rekognition Custom Labels based on training data and looked at the model accuracy, precision and F1 score for the resulting model.